In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

In [ ]:
sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    region = sagemaker_session.boto_session.region_name
    training_data_uri = "s3://{}/".format(AWSConfig.bucket)
    metric_definitions = [
        {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    ]


In [ ]:
estimator = PyTorch(entry_point='speach_emotion_recognition_vit.py',
                            hyperparameters=hyper_param,
                            role=role,
                            framework_version='1.2.0',
                            train_instance_count=2,
                            train_instance_type='ml.c5.xlarge')

In [ ]:
estimator.fit(training_data_uri, logs=True)

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')


In [ ]:
import numpy as np
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = predictor.predict(images.numpy())
        _, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
estimator.delete_endpoint()